In [18]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet1D(nn.Module):
    def __init__(self, in_channels, out_channels, nin_channels=32):
        super(UNet1D, self).__init__()

        # Encoder
        self.conv1 = nn.Conv1d(in_channels, nin_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(nin_channels, nin_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(nin_channels, nin_channels*2, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(nin_channels*2, nin_channels*2, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

      
        

        # Bottleneck
        self.bottleneck_conv1 = nn.Conv1d(nin_channels*2, nin_channels*2, kernel_size=3, padding=1)
        self.bottleneck_conv2 = nn.Conv1d(nin_channels*2, nin_channels*2, kernel_size=3, padding=1)

        # Decoder
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        #self.conv3 = nn.Conv1d(192, 64, kernel_size=3, padding=1)
        #self.conv4 = nn.Conv1d(64, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        # Encoder
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        x_pool1 = self.pool(x2)
        
        x3 = F.relu(self.conv3(x_pool1))
        x4 = F.relu(self.conv4(x3))
        x_pool = self.pool(x4)
        print(x_pool.shape)
        # Bottleneck
        x_bottleneck = F.relu(self.bottleneck_conv1(x_pool))
        x_bottleneck = F.relu(self.bottleneck_conv2(x_bottleneck))
        print(x_bottleneck.shape)
        print(x_pool1.shape)
        # Decoder
        x_upsample = self.upsample(x_bottleneck)
        #x_concat = torch.cat([x_upsample, x3], dim=1)
        #x3 = F.relu(self.conv3(x_concat))
        #x4 = self.conv4(x3)
        #x4=x_pool
        return x_pool1

# Example usage
model = UNet1D(in_channels=2, out_channels=2)
input_tensor = torch.randn(1, 2, 20) 

y=model(input_tensor)


torch.Size([1, 64, 5])
torch.Size([1, 64, 5])
torch.Size([1, 32, 10])


In [12]:
in_channels=2
nin_channels=32 
conv1 = nn.Conv1d(in_channels, nin_channels, kernel_size=3, padding=1)
conv2 = nn.Conv1d(nin_channels, nin_channels, kernel_size=3, padding=1)
conv3 = nn.Conv1d(nin_channels, nin_channels*2, kernel_size=3, groups=1,padding=1)
x=input_tensor
x1=conv1(x)
print(x1.shape)
x2=conv2(x1)
print(x2.shape)
x3=conv3(x2)
print(x3.shape)

torch.Size([1, 32, 20])
torch.Size([1, 32, 20])
torch.Size([1, 64, 20])


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet1D, self).__init__()

        # Encoder
        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(256, 512, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Bottleneck
        self.bottleneck_conv1 = nn.Conv1d(512, 1024, kernel_size=3, padding=1)
        self.bottleneck_conv2 = nn.Conv1d(1024, 1024, kernel_size=3, padding=1)

        # Decoder
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv5 = nn.Conv1d(1536, 512, kernel_size=3, padding=1)
        self.conv6 = nn.Conv1d(512, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv1d(256, 128, kernel_size=3, padding=1)
        self.conv8 = nn.Conv1d(128, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        # Encoder
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        x_pool1 = self.pool(x2)

        x3 = F.relu(self.conv3(x_pool1))
        x4 = F.relu(self.conv4(x3))
        x_pool2 = self.pool(x4)

        # Bottleneck
        x_bottleneck = F.relu(self.bottleneck_conv1(x_pool2))
        x_bottleneck = F.relu(self.bottleneck_conv2(x_bottleneck))

        # Decoder
        x_upsample1 = self.upsample(x_bottleneck)
        x_concat1 = torch.cat([x_upsample1, x4], dim=1)
        x5 = F.relu(self.conv5(x_concat1))
        x6 = F.relu(self.conv6(x5))

        x_upsample2 = self.upsample(x6)
        x_concat2 = torch.cat([x_upsample2, x2], dim=1)
        x7 = F.relu(self.conv7(x_concat2))
        x8 = self.conv8(x7)

        return x8

# Example usage
model = UNet1D(in_channels=2, out_channels=2)
input_tensor = torch.randn(1, 2, 64)  # Batch size, channels, sequence length
output = model(input_tensor)
print("Output shape:", output.shape)


RuntimeError: Given groups=1, weight of size [128, 256, 3], expected input[1, 384, 64] to have 256 channels, but got 384 channels instead